In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import subprocess
import importlib.util
import pandas as pd
import openai
import nest_asyncio
import asyncio

nest_asyncio.apply()

# Set OpenAI API key
with open('/content/drive/MyDrive/key.txt', 'r') as file:
    api_key = file.read().strip()
os.environ["OPENAI_API_KEY"] = api_key
openai.api_key = api_key

# Paths
folder_path = '/content/drive/MyDrive/Programs/'
csv_input_path = os.path.join(folder_path, 'program_rmse_results.csv')

# Check if results file exists and read existing results
if os.path.exists(csv_input_path):
    results_df = pd.read_csv(csv_input_path)
    evaluated_programs = set(results_df['code'].tolist())
else:
    results_df = pd.DataFrame(columns=['code', 'prompt', 'RMSE'])
    evaluated_programs = set()

# List all Python files in the folder
program_files = [f for f in os.listdir(folder_path) if f.endswith('.py')]

results = []

# Execute only unevaluated Python files
for file_name in program_files:
    if file_name in evaluated_programs:
        print(f"Skipping already evaluated program: {file_name}")
        continue

    file_path = os.path.join(folder_path, file_name)
    module_name = os.path.splitext(file_name)[0]
    prompt_file = module_name + '.txt'

    # Dynamically import module
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)

    # Execute CalculateRMSE and handle potential errors
    try:
        rmse = module.CalculateRMSE()
        rmse_numeric = float(rmse)
        results_df = pd.concat([results_df, pd.DataFrame([{'code': file_name, 'prompt': prompt_file, 'RMSE': rmse_numeric}])], ignore_index=True)
        print(f"Executed {file_name} successfully. RMSE: {rmse_numeric:.4e}")
    except Exception as e:
        print(f"Error executing {file_name}: {e}")

# Sort results by RMSE
results_df.sort_values(by='RMSE', inplace=True)

# Save sorted results to CSV on Google Drive
csv_output_path = os.path.join(folder_path, 'program_rmse_results.csv')
results_df.to_csv(csv_output_path, index=False, encoding='utf-8', float_format='%.4e')
print("Results saved to program_rmse_results.csv")

if not results_df.empty:
    best_program = results_df.iloc[0]
    print(f"\nBest Program: {best_program['code']} with RMSE: {best_program['RMSE']:.4e}")
else:
    print("\nNo successful executions.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Skipping already evaluated program: program1.py
Skipping already evaluated program: program2.py
Skipping already evaluated program: program3.py
Skipping already evaluated program: program4.py
Skipping already evaluated program: program5.py
Skipping already evaluated program: program6.py
RMSE: 1.11e-05
R² Score: 0.39
Executed program7.py successfully. RMSE: 1.1063e-05
Results saved to program_rmse_results.csv

Best Program: program1.py with RMSE: 1.0206e-05
